# Functions to scrape the Spotify API
These functions can be used to scrape the spotify API & generate the data needed from the Instagram Reels playlist.

In [2]:
import requests
import regex as re
import json
import time
import random

In [3]:
def get_spotify_token(client_id,client_secret):
        '''
        Get the spotify api access token using client id and secret from post request. 

        PARAMETERS
        ----------
        client_id: str
            client id for spotify developer account
        client_secret: str
            client secret for spotify developer account
        RETURNS
        -------
        str
            bearer access token for spotify api
        '''
        info = requests.post(url="https://accounts.spotify.com/api/token",
                        headers={"Content-Type":"application/x-www-form-urlencoded"},
                        data=f"grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}")
        return(info.json()['access_token'])

In [4]:
def parse_playlist_id(link='skyeler'):
    '''
    Get the spotify playlist id from a playlist link 

    PARAMETERS
    ----------
    link: str
        the link to a playlist on spotify
    
    RETURNS
    -------
    str
        the spotify playlist id
    '''
    regex_pattern = r'/playlist/([a-zA-Z0-9]+)'

    regex_match = re.search(regex_pattern,link)

    if regex_match:
        id = regex_match.group(1)
        return id
    return None

In [5]:
def get_title_info(headers,playlist_id):
    '''
    Makes a spotify api request to get the title and author of a playlist in a pretty string format.

    PARAMETERS
    ----------
    headers: dict
        api headers to use in the request, should be a dict with a bearer token.
    playlist_id: str
        the spotify unique id for a playlist

    RETURNS
    -------
    str
        the playlist name and ownner in the format "<playlist name> by <playlist author>"
    '''
    info = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}",headers=headers).json()
    return(f"{info['name']} by {info['owner']['display_name']}")

In [6]:
def parse_playlist(api_output):
    '''
    Parses the api output of a spotify api playlist tracks call. 
    Finds each artist per each song on the playlist and adds the artist to a dictionary, 
    with their spotify id and their api_link. 

    PARAMETERS
    ----------
    api_output: requests response type
        the output of an api get request

    RETURNS
    -------
    dict
        dictionary where the key is the name of an artist (str) and the value is a dictionary with info
    '''
    track_list = api_output.json()['items']
    output = {}

    for item in track_list:
        artist_info = item['track']['artists']
        for artist in artist_info:
            id = artist['id']
            name = artist['name']
            api_link = artist['href']
            output[name]={'id':id,
                        'api_link':api_link}
    return output

In [7]:
def query_playlist(spotify_token,playlist_link:str):
    '''
    Queries the playlist object to get the tracks.

    PARAMETERS
    ----------
    playlist_link: str
        spotify api link to get the tracks from a playlist
    headers: dict
        api headers to use in the request, should be a dict with a bearer token.

    RETURNS
    -------
    request object
    '''
    headers={"Authorization": f"Bearer {spotify_token}"} #set the spotify headers
    playlist_id = parse_playlist_id(playlist_link)
    #title_info = get_title_info(headers,playlist_id)
    playlist = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks",headers=headers) #get the tracks
    return playlist

In [8]:
def get_artist_info(link,headers):
   '''
   Makes a request based on an artist's link and returns a dictionary with additional 
   info about that artist. 

   PARAMETERS
   ----------
   link: str
        spotify api link to get information about an artist
    headers: dict
        api headers to use in the request, should be a dict with a bearer token.

   RETURNS
   -------
   dict
        a dictionary with the artist's genres, image information, and popularity
   '''
   info = requests.get(link,headers=headers)
   print(info)
   info = info.json()
   print(info)
   try:
     genres = info['genres']
   except:
      genres = []
      print(info)
   try:
     followers = info['followers']['total']
   except:
      followers = None
   try:
     img_info = info['images']
   except:
      img_info = []
   try:
     popularity = info['popularity']
   except:
      popularity = None
   return {'genres':genres,
           'img_info':img_info,
           'popularity':popularity,
           'followers':followers}

In [9]:
def get_collaborators(albums:dict)->list:
    '''Gets the collaborators of an artist'''
    collaborators = {''}
    for al in albums['items']:
        artist_list = al['artists']
        for artist in artist_list:
            collaborators.add((artist['name'],artist['uri']))
    collaborators.remove('')
    return(list(collaborators))

In [10]:
def get_collaborators_number(albums:dict)->list:
    '''Gets the collaborators of an artist'''
    collaborators = {}
    for al in albums['items']:
        artist_list = al['artists']
        for artist in artist_list:
            if artist['name'] not in collaborators.keys():
                collaborators[artist['name']]={'count':1,'uri':artist['uri']}
            else:
                collaborators[artist['name']]['count']+=1
    return(collaborators)

In [11]:
def save_file(data:dict,filepath:str='cache.json')->None:
    '''Saves a json file'''
    dumped_json_cache = json.dumps(data)
    fw = open(filepath,"w")
    fw.write(dumped_json_cache)
    fw.close()
    return

In [12]:
def open_file(filepath:str='cache.json')->dict:
    '''Opens a file'''
    cache_file = open(filepath, 'r')
    cache_contents = cache_file.read()
    artist_data = json.loads(cache_contents)
    cache_file.close()
    return artist_data

In [13]:
def get_random_artist(headers):
    '''Returns a random artist
    Modified from https://perryjanssen.medium.com/getting-random-tracks-using-the-spotify-api-61889b0c0c27

    Input:
        headers: dict with header info
    '''
    chars = '''
    abcdefghijklmnopqrstuvwxyz1234567890!.@?#$%^&*()'"/><{}[]\|`~_-+=
    '''
    rand_char = random.choice(list(chars))
    if round(random.uniform(0, 1)) ==0:
        query = '%'+rand_char
    else:
        query=rand_char+'%'
    offset = round(random.uniform(0, 1000))
    info = requests.get(f"https://api.spotify.com/v1/search?q={query}&type=artist&offset={str(offset)}",headers=headers)
    info = info.json()
    print(info)
    top = info['artists']['items'][0]
    try:
        genres = top['genres']
    except:
        genres = []
    try:
        followers = top['followers']['total']
    except:
        followers = None
    try:
        img_info = top['images']
    except:
        img_info = []
    try:
        popularity = top['popularity']
    except:
        popularity = None
    return {'name':top['name'],
            'genres':genres,
            'img_info':img_info,
            'popularity':popularity,
            'followers':followers,
            'api_link':top['external_urls']['spotify']}

In [24]:
# get the token
tok1, tok2 = 
DATE = '11_08' 
spotify_token = get_spotify_token(tok1,tok2)
headers={"Authorization": f"Bearer {spotify_token}"}
# query the playlist
playlist_ex = query_playlist(spotify_token,'https://open.spotify.com/playlist/6dSz0AWxkgBfhlNBFyzqFf')
playlist_dict = parse_playlist(playlist_ex)
# open cache
cache = open_file('cache.json')

# initialize output
output = {}

# for each celeb in the playlist
for celeb in playlist_dict.keys():
#       check if in cache.keys
    if celeb in cache.keys():
        output[celeb]=cache[celeb]
        continue # no need to do more stuff
#       add data to output
    artist_metadata = get_artist_info(playlist_dict[celeb]['api_link'],headers)
    albums = requests.get(playlist_dict[celeb]['api_link']+'/albums',headers=headers).json()
    artist_metadata['collaborators']=get_collaborators_number(albums)
    artist_metadata['in_playlist']=True
    # update output & cache
    output[celeb]=artist_metadata
    cache[celeb]=artist_metadata

# update cache
save_file(cache,'cache.json')
# save outpput
save_file(output,f'{DATE}_1.json')


<Response [200]>
{'external_urls': {'spotify': 'https://open.spotify.com/artist/3eVa5w3URK5duf6eyVDbu9'}, 'followers': {'href': None, 'total': 7766971}, 'genres': ['k-pop'], 'href': 'https://api.spotify.com/v1/artists/3eVa5w3URK5duf6eyVDbu9', 'id': '3eVa5w3URK5duf6eyVDbu9', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb727a1f1f508238a20ac9fdbf', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174727a1f1f508238a20ac9fdbf', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178727a1f1f508238a20ac9fdbf', 'height': 160, 'width': 160}], 'name': 'ROSÉ', 'popularity': 84, 'type': 'artist', 'uri': 'spotify:artist:3eVa5w3URK5duf6eyVDbu9'}


In [28]:
# want to meta data on the people NOT in the playlist
non_playlist_copy = output.copy()
counter = 0
for item in output.keys(): # for each top 100 artist
    time.sleep(1)
    for name in non_playlist_copy[item]['collaborators']: # for each collaborator
        print(name)
        time.sleep(1)
        counter +=1
        print(counter)
        #name = x[0]
        # if it isn't in the output & playlist copy
        if name not in output.keys() and name not in non_playlist_copy.keys():
            print(name)
            # check if it is in the cache
            if name in cache.keys():
                artist_metadata = cache[name]
                artist_metadata['in_playlist']=False
                non_playlist_copy[name] = artist_metadata
                continue

            uri = non_playlist_copy[item]['collaborators'][name]['uri'].split(':')[2]
            try:
                artist_metadata = get_artist_info(f'https://api.spotify.com/v1/artists/{uri}',headers)
                albums = requests.get(f'https://api.spotify.com/v1/artists/{uri}'+'/albums?offset=0&limit=50',headers=headers).json()
                artist_metadata['collaborators']=get_collaborators_number(albums)

                artist_metadata['in_playlist']=False # this is false now
                non_playlist_copy[name]=artist_metadata
            except:
                spotify_token = get_spotify_token(tok1,tok2)
                headers={"Authorization": f"Bearer {spotify_token}"}
                artist_metadata = get_artist_info(f'https://api.spotify.com/v1/artists/{uri}',headers)
                albums = requests.get(f'https://api.spotify.com/v1/artists/{uri}'+'/albums?offset=0&limit=50',headers=headers).json()
                artist_metadata['collaborators']=get_collaborators_number(albums)

            artist_metadata['in_playlist']=False # this is false now
            non_playlist_copy[name]=artist_metadata
            #update cache
            cache[name]=artist_metadata

# update cache
save_file(cache,'cache.json')
# save outpput
save_file(non_playlist_copy,f'{DATE}_2.json')

ROSÉ
1
Bruno Mars
2
G-DRAGON
3
G-DRAGON
BLACKPINK
4
BLACKPINK
Various Artists
5
Various Artists
Bruno Mars
6
Anderson .Paak
7
Anderson .Paak
Silk Sonic
8
Silk Sonic
Lady Gaga
9
Ed Sheeran
10
Ed Sheeran
Chris Stapleton
11
Chris Stapleton
Cardi B
12
Cardi B
Gucci Mane
13
Gucci Mane
Kodak Black
14
Kodak Black
James Hype
15
James Hype
David Guetta
16
David Guetta
R3HAB
17
R3HAB
Alan Walker
18
PARTYNEXTDOOR
19
PARTYNEXTDOOR
BLVK JVCK
20
BLVK JVCK
Lil Wayne
21
Lil Wayne
B.o.B
22
B.o.B
Bad Meets Evil
23
Bad Meets Evil
Travie McCoy
24
Travie McCoy
Snoop Dogg
25
Snoop Dogg
Wiz Khalifa
26
Wiz Khalifa
Various Artists
27
Summer Hits
28
Summer Hits
Major Lazer
29
Major Lazer
Eminem
30
Eminem
Billie Eilish
31
Charli xcx
32
ROSALÍA
33
ROSALÍA
Justin Bieber
34
Justin Bieber
Khalid
35
Khalid
Sofi Tukker
36
Sofi Tukker
Vince Staples
37
Vince Staples
Selena Gomez
38
Selena Gomez
OneRepublic
39
OneRepublic
YUNGBLUD
40
YUNGBLUD
ヴァリアス・アーティスト
41
ヴァリアス・アーティスト
Various Artists
42
Hans Zimmer
43
Hans Zimmer
Sabr

## Now we need to randomly sample 100 ish artists

In [35]:
## data
tok1, tok2 = 
spotify_token = get_spotify_token(tok1,tok2)
headers={"Authorization": f"Bearer {spotify_token}"}

In [36]:
# I want to get the same number of artists as the playlist
playlist_data_len = len(open_file('11_01_playlist_layer1.json').keys())

random_artists = {}
while len(random_artists.keys()) < playlist_data_len:
    time.sleep(.75)
    try:
        art = get_random_artist(headers)
        random_artists[art['name']]=art
    except:
        continue

{'artists': {'href': 'https://api.spotify.com/v1/search?query=%25%7E&type=artist&offset=137&limit=20', 'items': [], 'limit': 20, 'next': None, 'offset': 137, 'previous': 'https://api.spotify.com/v1/search?query=%25%7E&type=artist&offset=117&limit=20', 'total': 0}}
{'artists': {'href': 'https://api.spotify.com/v1/search?query=b%25&type=artist&offset=231&limit=20', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0GWCNkPi54upO9WLlwjAHd'}, 'followers': {'href': None, 'total': 2648546}, 'genres': ['boy band', 'pop', 'post-teen pop'], 'href': 'https://api.spotify.com/v1/artists/0GWCNkPi54upO9WLlwjAHd', 'id': '0GWCNkPi54upO9WLlwjAHd', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eb0b5d1b1e2367fff6b7936920', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab676161000051740b5d1b1e2367fff6b7936920', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f1780b5d1b1e2367fff6b7936920', 'width': 160}], 'name': '

In [37]:
# open cache
cache = open_file('cache.json')

# initialize output
output = {}

# for each celeb in the playlist
for celeb in random_artists.keys():
#       check if in cache.keys
    if celeb in cache.keys():
        output[celeb]=cache[celeb]
        continue # no need to do more stuff
#       add data to output
    artist_metadata = random_artists[celeb]
    uri = random_artists[celeb]['api_link'].split('/')[-1]
    # print(random_artists[celeb]['api_link']+'/albums')
    albums = requests.get(f'https://api.spotify.com/v1/artists/{uri}'+'/albums?offset=0&limit=50',headers=headers).json()
    artist_metadata['collaborators']=get_collaborators_number(albums)
    artist_metadata['in_playlist']=False
    # update output & cache
    output[celeb]=artist_metadata
    cache[celeb]=artist_metadata

# update cache
save_file(cache,'cache.json')
# save outpput
#DATE = 
save_file(output,f'{DATE}_random_1.json')

In [38]:
# output=open_file(
#     '10_26_random_1.json'
# )
cache = open_file('cache.json')
DATE = '11_01_random_2'

In [39]:
# want to meta data on the people NOT in the playlist
tok1, tok2 = 
spotify_token = get_spotify_token(tok1,tok2)
headers={"Authorization": f"Bearer {spotify_token}"}
non_playlist_copy = output.copy()
counter = 0
for item in output.keys(): # for each top 100 artist
    #time.sleep(1)
    for name in non_playlist_copy[item]['collaborators']: # for each collaborator
        print(name)
        #time.sleep(1)
        counter +=1
        print(counter)
        #name = x[0]
        # if it isn't in the output & playlist copy
        if name not in output.keys() and name not in non_playlist_copy.keys():
            print(name)
            # check if it is in the cache
            if name in cache.keys():
                artist_metadata = cache[name]
                artist_metadata['in_playlist']=False
                non_playlist_copy[name] = artist_metadata
                continue

            uri = non_playlist_copy[item]['collaborators'][name]['uri'].split(':')[2]
            try:
                time.sleep(2)
                artist_metadata = get_artist_info(f'https://api.spotify.com/v1/artists/{uri}',headers)
                albums = requests.get(f'https://api.spotify.com/v1/artists/{uri}'+'/albums?offset=0&limit=50',headers=headers).json()
                artist_metadata['collaborators']=get_collaborators_number(albums)

                artist_metadata['in_playlist']=False # this is false now
                non_playlist_copy[name]=artist_metadata
            except:
                spotify_token = get_spotify_token(tok1,tok2)
                headers={"Authorization": f"Bearer {spotify_token}"}
                artist_metadata = get_artist_info(f'https://api.spotify.com/v1/artists/{uri}',headers)
                albums = requests.get(f'https://api.spotify.com/v1/artists/{uri}'+'/albums?offset=0&limit=50',headers=headers).json()
                artist_metadata['collaborators']=get_collaborators_number(albums)

            artist_metadata['in_playlist']=False # this is false now
            save_file(cache,'cache.json')
            non_playlist_copy[name]=artist_metadata
            #update cache
            cache[name]=artist_metadata

# update cache
save_file(cache,'cache.json')
# save outpput
save_file(non_playlist_copy,f'{DATE}_random_2.json')

Big Time Rush
1
Maffio
2
Maffio
<Response [200]>
{'external_urls': {'spotify': 'https://open.spotify.com/artist/5RzT7CM6Ot0sh0EHefMicV'}, 'followers': {'href': None, 'total': 27883}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/5RzT7CM6Ot0sh0EHefMicV', 'id': '5RzT7CM6Ot0sh0EHefMicV', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb3cd1c22d5192d0c55971e3cf', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab676161000051743cd1c22d5192d0c55971e3cf', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f1783cd1c22d5192d0c55971e3cf', 'height': 160, 'width': 160}], 'name': 'Maffio', 'popularity': 56, 'type': 'artist', 'uri': 'spotify:artist:5RzT7CM6Ot0sh0EHefMicV'}
Calacote
3
Calacote
<Response [200]>
{'external_urls': {'spotify': 'https://open.spotify.com/artist/52PhGA4a6erATMkWfLuFvO'}, 'followers': {'href': None, 'total': 5819}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/52PhGA4a6erATMkWfLuFvO', 'id': '52PhGA4a6e

In the cells below, you want to combine the graph.

In [16]:
import networkx as nx
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [14]:
playlist_network = open_file('11_01_playlist_layer2.json')
random_network = open_file('11_01_random_2.json')

In [17]:
# combine!!

# playlist network doesn't have name key, my bad
for artist in playlist_network.keys():
    playlist_network[artist]['name']=artist

g = nx.Graph()
for artist in playlist_network.keys():
    g.add_node(artist)
    colab = playlist_network[artist]['collaborators'].keys()
    for y in colab:
        if artist !=y:
            if y not in g.nodes:
                g.add_node(y)
            g.add_edge(artist,y)

for n in g.nodes():
    g.nodes[n]['source'] = 'playlist'
    if n in playlist_network.keys():
        for key in playlist_network[n].keys():
            g.nodes[n][key]= playlist_network[n][key]
    else:
        g.nodes[n]['in_playlist']=False
        g.nodes[n]['name']=n

for artist in random_network.keys():
    if artist not in g.nodes:
        g.add_node(artist)
    colab = random_network[artist]['collaborators'].keys()
    for y in colab:
        if artist !=y:
            if y not in g.nodes:
                g.add_node(y)
            g.add_edge(artist,y)

for n in g.nodes():
    if n in random_network.keys():
        for key in random_network[n].keys():
            g.nodes[n][key]= random_network[n][key]
        g.nodes[n]['in_playlist']=False
        g.nodes[n]['name']=n
        if n not in playlist_network.keys():
            g.nodes[n]['source']='random'

In [55]:
pd.DataFrame.from_dict(g.nodes, orient='index').isna().sum()

source             367
genres           10883
img_info         10883
popularity       10883
followers        10883
collaborators    10883
in_playlist          0
link             15138
name               351
api_link         15111
dtype: int64

In [18]:
tok1, tok2 = 
spotify_token = get_spotify_token(tok1,tok2)
headers={"Authorization": f"Bearer {spotify_token}"}

In [19]:
missing = 0
for i in g.nodes:
    if'genres' not in g.nodes[i].keys():
        missing +=1
missing

30568

In [81]:
g.number_of_nodes()

34291

In [84]:
g.nodes['Hot Chelle Rae']

{'source': 'playlist',
 'in_playlist': False,
 'name': 'Hot Chelle Rae',
 'genres': ['dance rock', 'neon pop punk', 'post-teen pop'],
 'img_info': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba4ed9ad0da4d1801ca1683df',
   'height': 640,
   'width': 640},
  {'url': 'https://i.scdn.co/image/ab67616100005174a4ed9ad0da4d1801ca1683df',
   'height': 320,
   'width': 320},
  {'url': 'https://i.scdn.co/image/ab6761610000f178a4ed9ad0da4d1801ca1683df',
   'height': 160,
   'width': 160}],
 'popularity': 56,
 'followers': 559101}

In [23]:
cache = open_file('cache.json')
counter = 0
for x in g.nodes:
    counter +=1
    if counter % 169 ==0:
        tok1, tok2 = 
        spotify_token = get_spotify_token(tok1,tok2)
        headers={"Authorization": f"Bearer {spotify_token}"}
    if 'genres' not in g.nodes[x].keys():
        if (x in cache.keys()):
            print(g.nodes[x])
            for key in cache[x].keys():
                g.nodes[x][key]= cache[x][key]
            continue
        print(x)
        time.sleep(2)
        print(counter)
        parent_node = list(g.edges(x))[0][1]
        if parent_node in random_network.keys():
            uri = random_network[parent_node]['collaborators'][x]['uri'].split(':')[2]
        else:
            uri = playlist_network[parent_node]['collaborators'][x]['uri'].split(':')[2]
        artist_metadata = get_artist_info(f'https://api.spotify.com/v1/artists/{uri}',headers)

        for key in artist_metadata.keys():
            g.nodes[x][key]= artist_metadata[key]
        cache[x]=artist_metadata
        cache['in_playlist']=False
        save_file(cache,'cache.json')
        nx.write_pajek(g, "11_01_total.net")

save_file(cache,'cache.json')
nx.write_pajek(g, "11_01_total.net")

Mohammed Irfan
2104
<Response [200]>
{'external_urls': {'spotify': 'https://open.spotify.com/artist/6QtJ6BVdj43aDpgjnVp2tY'}, 'followers': {'href': None, 'total': 2598183}, 'genres': ['filmi', 'modern bollywood'], 'href': 'https://api.spotify.com/v1/artists/6QtJ6BVdj43aDpgjnVp2tY', 'id': '6QtJ6BVdj43aDpgjnVp2tY', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5ebb074074cf866ba3b3ebc09a6', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174b074074cf866ba3b3ebc09a6', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178b074074cf866ba3b3ebc09a6', 'height': 160, 'width': 160}], 'name': 'Mohammed Irfan', 'popularity': 64, 'type': 'artist', 'uri': 'spotify:artist:6QtJ6BVdj43aDpgjnVp2tY'}


/Users/Skyeler/anaconda3/envs/si649f23/lib/python3.11/site-packages/networkx/readwrite/pajek.py:75: UserWarning: Node attribute genres is not processed. Non-string attribute.
  warnings.warn(
/Users/Skyeler/anaconda3/envs/si649f23/lib/python3.11/site-packages/networkx/readwrite/pajek.py:75: UserWarning: Node attribute img_info is not processed. Non-string attribute.
  warnings.warn(
/Users/Skyeler/anaconda3/envs/si649f23/lib/python3.11/site-packages/networkx/readwrite/pajek.py:75: UserWarning: Node attribute popularity is not processed. Non-string attribute.
  warnings.warn(
/Users/Skyeler/anaconda3/envs/si649f23/lib/python3.11/site-packages/networkx/readwrite/pajek.py:75: UserWarning: Node attribute followers is not processed. Non-string attribute.
  warnings.warn(
/Users/Skyeler/anaconda3/envs/si649f23/lib/python3.11/site-packages/networkx/readwrite/pajek.py:75: UserWarning: Node attribute collaborators is not processed. Non-string attribute.
  warnings.warn(
/Users/Skyeler/anaconda3

Mukhtar Sahota
2105
<Response [200]>
{'external_urls': {'spotify': 'https://open.spotify.com/artist/5SxQsyAuCp6fIqF3eGoirU'}, 'followers': {'href': None, 'total': 9262}, 'genres': ['classic punjabi pop'], 'href': 'https://api.spotify.com/v1/artists/5SxQsyAuCp6fIqF3eGoirU', 'id': '5SxQsyAuCp6fIqF3eGoirU', 'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b273c37fad084d7a29488a44a92c', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616d00001e02c37fad084d7a29488a44a92c', 'height': 300, 'width': 300}, {'url': 'https://i.scdn.co/image/ab67616d00004851c37fad084d7a29488a44a92c', 'height': 64, 'width': 64}], 'name': 'Mukhtar Sahota', 'popularity': 47, 'type': 'artist', 'uri': 'spotify:artist:5SxQsyAuCp6fIqF3eGoirU'}
Avadhoot Gupte
2106
<Response [200]>
{'external_urls': {'spotify': 'https://open.spotify.com/artist/6mxY3ekITToaEK2XGtaock'}, 'followers': {'href': None, 'total': 778759}, 'genres': ['filmi'], 'href': 'https://api.spotify.com/v1/artists/6mxY3ekITToaEK2XGta

KeyboardInterrupt: 